### Section 1: Preparation

In [1]:
##########################################################################################################
#
#         2023-5 v1: Base version
#         2023-5 v2: Calculates all years data of revenue, eps and freeCashFlow based on growth number
#         2023-6 v3: Add visualization to view data 
#
##########################################################################################################

import pandas as pd
import numpy as np
from datetime import date

import yfinance as yf
import fundamentalanalysis as fa

import plotly.express as px
import plotly.graph_objects as go


In [2]:
# Input section. Enter ticker, start_date (normally first IPO date) and end_date (normally last year)
ticker = 'AAPL'
api_key = "99468a6d827666a6ee01f6788171dc40"

start_date = '1965-01-01'
end_date = '2022-12-31'

In [3]:
# Write single dataframe to excel sheet
def write_a_dataframe_to_excel(df, filename, sheetname):
    with pd.ExcelWriter(f'{filename}.xlsx', 
                        engine='xlsxwriter', 
                        datetime_format='yyyy') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheetname)
        writer.sheets[sheetname] = worksheet

        COLUMN = 0
        row = 0

        worksheet.write_string(row, COLUMN, df.name)
        row += 1
        df.to_excel(writer, sheet_name=sheetname,
                    startrow=row, startcol=COLUMN)
        
# Automatically write multiple dataframe to excel sheet
def write_dataframes_to_excel(dataframes, filename, sheetname):
    with pd.ExcelWriter(f'{filename}.xlsx', 
                        engine='xlsxwriter',
                        datetime_format='yyyy') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheetname)
        writer.sheets[sheetname] = worksheet

        COLUMN = 0
        row = 0

        for df in dataframes:
            worksheet.write_string(row, COLUMN, df.name)
            row += 1
            df.to_excel(writer, sheet_name=sheetname,
                        startrow=row, startcol=COLUMN,
                       float_format = '%.2f')
            row += df.shape[0] + 2

# Apply percentage format to number
def percentage_format(x):
    if pd.isna(x):
        return str(x)
    else:
        return f"{x:.2%}"

# Update NaN value of revenue, eps and freeCashFlow based on growth calculation
# Reason: FinancialModellingPrep's API only provide 5 years data for free account. But provides all years data for growth.
# This function calculates all years data of revenue, eps and freeCashFlow based on growth
def update_per_growth(df, rownum):
    for col in range(1, len(df.columns)):
        if pd.isna(df.iloc[rownum][col]):
            df.iloc[rownum, col] = df.iloc[rownum][col-1] / (df.iloc[rownum+1][col-1]+1)
            
    return df

In [4]:
stock = yf.download(ticker, start_date, end_date)      # get stock history data up to last year
stock_today = yf.download(ticker, end_date)            # get stock data for current year

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Section 2: Get price info

In [5]:
# Obtain last day in Dec. of each year
stock['year'] = stock.index.year
stock['month'] = stock.index.month
stock['day'] = stock.index.day
max_day_of_year = stock.loc[stock.index.month == 12].groupby('year').max()['day']

# Get historical stock price per year (year end price)
stock_close = stock.loc[(stock.index.month == 12) & (stock.index.day == max_day_of_year[stock.index.year])]
# stock_close = pd.concat([stock_close,stock_today])
stock_close = pd.concat([stock_close, stock_today.loc[stock_today.iloc[-1].name.strftime("%Y-%m-%d"):]])
stock_price = pd.DataFrame()
stock_price['Price'] = stock_close['Close']

# Formatting
stock_price = stock_price.sort_index(ascending=False).transpose()

# Get price percentage changes per year
returns = pd.DataFrame()
returns['PriceGrowth'] = stock_close['Close'].pct_change()

# Formatting
returns['PriceGrowth'] = returns['PriceGrowth'].apply(lambda x: percentage_format(x))
returns = returns.sort_index(ascending=False).transpose()

# Combine 2 rows
price_info = pd.concat([stock_price, returns])

# Change column names from timestamp format to 4 digit year
price_info.rename(columns=lambda s: str(s)[:4], inplace=True)
price_info

Date,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
Price,184.919998,129.929993,177.570007,132.690002,73.412498,39.435001,42.307499,28.955,26.315001,27.594999,...,0.314732,0.359375,0.375,0.180804,0.098214,0.130022,0.108817,0.133371,0.098772,0.152344
PriceGrowth,42.32%,-26.83%,33.82%,80.75%,86.16%,-6.79%,46.11%,10.03%,-4.64%,37.72%,...,-12.42%,-4.17%,107.41%,84.09%,-24.46%,19.49%,-18.41%,35.03%,-35.17%,nan


In [6]:
# Transpose price_info for line chart ploting
price_info_tr = price_info.sort_index(axis=1).transpose()

# Convert 'PriceGrowth' to float variable
price_info_tr['PriceGrowth'] = price_info_tr['PriceGrowth'].apply(lambda x: 0 if pd.isna(x) else float(x.strip('%'))/100)

In [7]:
# Plot line chart for Price & Price Growth using graph_objects
fig = go.Figure()

# Add Price trace
fig.add_trace(go.Scatter(
    x=price_info_tr.index.values, y=price_info_tr['Price'],
    name='Price', yaxis='y'
))

# Add PriceGrowth trace
fig.add_trace(go.Scatter(
    x=price_info_tr.index.values, y=price_info_tr['PriceGrowth'],
    name='Price Growth', mode='lines', yaxis='y2'
))

# Update layout
fig.update_layout(
    title=ticker + ' stock Price and Price Growth',
    width=1000,
    yaxis=dict(title='Price'),
    yaxis2=dict(title='Price Growth', overlaying='y', side='right', tickformat = ",.0%"),
    legend=dict(x=0.85, y=1.2, traceorder='normal')
)

# Show the fig & export chart
fig.show()
fig.write_image("images/" + ticker + " price chart.png")

### Section 3: Bring all finanical data needed for instrinsic value calculation

In [8]:
# Loan other finanical data
income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
# dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
entreprise_value = fa.enterprise(ticker, api_key)
balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")

# Store financial data in results
results = pd.DataFrame()
results = pd.concat([income_statement_annually.loc['revenue'],
                     growth_annually.loc['revenueGrowth'], 
                     income_statement_annually.loc['netIncome'],
                     income_statement_annually.loc['eps'],
                     growth_annually.loc['epsgrowth'],
                     key_metrics_annually.loc['bookValuePerShare'],
                     growth_annually.loc['bookValueperShareGrowth'],
                     cash_flow_statement_annually.loc['freeCashFlow'],
                     growth_annually.loc['freeCashFlowGrowth'],
                     key_metrics_annually.loc['freeCashFlowPerShare'],                     
                     key_metrics_annually.loc['marketCap'],
                     key_metrics_annually.loc['enterpriseValue'],
                     key_metrics_annually.loc['peRatio'],
                     key_metrics_annually.loc['pbRatio'],
                     key_metrics_annually.loc['roe'],
                     balance_sheet_annually.loc['cashAndShortTermInvestments'],
                     balance_sheet_annually.loc['totalDebt'],
                     entreprise_value.loc['numberOfShares']
#                      dcf_annually.loc['DCF']
                    ], axis=1)

# Function to calculate eeRatio = enterpriseValue/netIncome
def calc_eeRatio(df):
    df['eeRatio'] = df['enterpriseValue'] / df['netIncome']
    return df

# Update eeRatio
results.rename({'pbRatio': 'eeRatio'}, axis='columns', inplace=True)
results = calc_eeRatio(results).transpose()

# results = results.transpose()
results = pd.concat([price_info, results])

# Format decimal fields format as percentage
results.loc['revenueGrowth'] = results.loc['revenueGrowth'].apply(lambda x: percentage_format(x))
results.loc['epsgrowth'] = results.loc['epsgrowth'].apply(lambda x: percentage_format(x))
results.loc['bookValueperShareGrowth'] = results.loc['bookValueperShareGrowth'].apply(lambda x: percentage_format(x))
results.loc['freeCashFlowGrowth'] = results.loc['freeCashFlowGrowth'].apply(lambda x: percentage_format(x))
results.loc['roe'] = results.loc['roe'].apply(lambda x: percentage_format(x))

In [9]:
# Data Preparation for plotly charts
# Transpose price_info for line chart ploting
results_tr = results.sort_index(axis=1).transpose()

# Convert PriceGrowth/revenueGrowth/epsgrowth/freeCashFlowGrowth to float variable
results_tr['PriceGrowth'] = results_tr['PriceGrowth'].apply(lambda x: float(x) if pd.isna(x) else float(x.strip('%'))/100)
results_tr['revenueGrowth'] = results_tr['revenueGrowth'].apply(lambda x: float(x) if pd.isna(x) else float(x.strip('%'))/100)
results_tr['epsgrowth'] = results_tr['epsgrowth'].apply(lambda x: float(x) if pd.isna(x) else float(x.strip('%'))/100)
results_tr['freeCashFlowGrowth'] = results_tr['freeCashFlowGrowth'].apply(lambda x: float(x) if pd.isna(x) else float(x.strip('%'))/100)

In [10]:
# Plot line chart for fundamental Growth using graph_objects
fig = go.Figure()

# Add PriceGrowth trace
fig.add_trace(go.Scatter(
    x=results_tr.index.values, y=results_tr['PriceGrowth'],
    name='PriceGrowth', yaxis='y'
))

# Add revenueGrowth trace
fig.add_trace(go.Scatter(
    x=results_tr.index.values, y=results_tr['revenueGrowth'],
    name='revenueGrowth', yaxis='y'
))

# Add epsgrowth trace
fig.add_trace(go.Scatter(
    x=results_tr.index.values, y=results_tr['epsgrowth'],
    name='epsgrowth', yaxis='y'
))

# Add freeCashFlowGrowth trace
fig.add_trace(go.Scatter(
    x=results_tr.index.values, y=results_tr['freeCashFlowGrowth'],
    name='freeCashFlowGrowth', yaxis='y2'
))

# Update layout
fig.update_layout(
    title=ticker + ' stock Growth',
    width=1000,
    yaxis=dict(title='Price/Revenue/EPS Growth', tickformat = ",.0%"),
    yaxis2=dict(title='Freecashflow Growth', overlaying='y', side='right', tickformat = ",.0%"),
    legend=dict(x=0.78, y=1.32, traceorder='normal')
)

# Show the fig & export chart
fig.show()
fig.write_image("images/" + ticker + " fundamental growth chart.png")

### Section 4: Compound Annual Growth Rate (CAGR) for Price, Revenue, EPS & Free Cash Flow

In [11]:
# Calc Compound Annual Growth Rate (CAGR)
def CAGR(first, last, periods):
    result = 0
    
    if first > 0 and last > 0:
        result = (last/first)**(1/periods)-1
    elif first < 0 and last < 0:
        result = -1 * ((abs(last)/abs(first))**(1/periods)-1)
    elif first < 0 and last > 0:
        result = ((last+2*abs(first))/abs(first))**(1/periods)-1
    elif first > 0 and last < 0:
        result = -1 * (((abs(last)+2*first)/first)**(1/periods)-1)
        
    return result

# Get last not null column
def earliest_col_notna(df,idx):
    for col in reversed(df.columns):
        if pd.notna(df.loc[idx][col]):
            return df.columns.get_loc(col)
        
# Calc CAGR (5/10/All years) for growth related columns
def calc_CAGR_for_col(df, idx):
    earliest_year = earliest_col_notna(df, idx)
    
    five_yrs_CAGR = np.nan
    ten_yrs_CAGR = np.nan
    All_CAGR = np.nan
    
    if earliest_year < 6:
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
    elif earliest_year < 11:
        five_yrs_CAGR = CAGR(df.loc[idx][6], df.loc[idx][1], 5)
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
    else:
        five_yrs_CAGR = CAGR(df.loc[idx][6], df.loc[idx][1], 5)
        ten_yrs_CAGR = CAGR(df.loc[idx][11], df.loc[idx][1], 10)
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
        
    return [five_yrs_CAGR, ten_yrs_CAGR, All_CAGR]

In [12]:
# Calc average returns from growth related columns
returns_lists = []

returns_lists.append(calc_CAGR_for_col(results,'Price'))
returns_lists.append(calc_CAGR_for_col(results,'revenue'))
returns_lists.append(calc_CAGR_for_col(results,'eps'))
returns_lists.append(calc_CAGR_for_col(results,'freeCashFlow'))

avg_returns = pd.DataFrame(returns_lists, 
             index=['PriceGrowth','revenueGrowth','epsgrowth','freeCashFlowGrowth'], 
             columns=['5 years Avg','10 years Avg','All years Avg'])

In [13]:
# Create the figure
fig = go.Figure()

for i, column in enumerate(avg_returns.columns):
    fig.add_trace(go.Bar(
        x=avg_returns.index.values,
        y=avg_returns[column],
        name=column,
        offsetgroup=i,
        # marker_color=f"rgba({(i*70)%256},{(i*120)%256},{(i*200)%256},0.6)"  # Custom color for each bar
    ))

# Update the layout
fig.update_layout(
    title=ticker + ' Compound Annual Growth Rate (CAGR)',
    yaxis=dict(title='Annual Growth', tickformat = ",.0%"),
    barmode='group', width=800
)

# Show the figure & export chart
fig.show()
fig.write_image("images/" + ticker + " Compound Annual Growth Rate.png")

### Last section: Put all data together and save to excel

In [284]:
# Final Formatting
avg_returns['5 years Avg'] = avg_returns['5 years Avg'].apply(lambda x: str(x) if pd.isna(x) else f"{x:.2%}")
avg_returns['10 years Avg'] = avg_returns['10 years Avg'].apply(lambda x: str(x) if pd.isna(x) else f"{x:.2%}")
avg_returns['All years Avg'] = avg_returns['All years Avg'].apply(lambda x: str(x) if pd.isna(x) else f"{x:.2%}")

# Set report section titles
results.name = ticker + ' Financial data'
avg_returns.name = ticker + ' Compound Annual Growth Rate (CAGR)'

dataframes = (results, avg_returns)
write_dataframes_to_excel(dataframes, 
                          'data\Stock fundamental analysis - '+ ticker + ' ' + date.today().strftime("%Y-%m"), 
                          ticker)